In [20]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchmetrics import Precision, Recall

from torchvision.datasets import ImageFolder
from torchvision import transforms

from torchmetrics import Accuracy

In [21]:
from zipfile import ZipFile 

# loading the temp.zip and creating a zip object 
with ZipFile("data/electricity_consump.zip", 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
    zObject.extractall(path=".") 

## Generating sequences

##### To be able to train neural networks on sequential data, you need to pre-process it first. You'll chunk the data into inputs-target pairs, where the inputs are some number of consecutive data points and the target is the next data point.

##### Your task is to define a function to do this called create_sequences(). As inputs, it will receive data stored in a DataFrame, df and seq_length, the length of the inputs. As outputs, it should return two NumPy arrays, one with input sequences and the other one with the corresponding targets.

##### As a reminder, here is how the DataFrame df looks like:

<table>
<tr><th></th>                 <th>timestamp</th>  <th>consumption</th></tr>
<tr><td>0</td>      <td>2011-01-01 00:15:00</td>    <td>-0.704319</td></tr>
<tr><td>...</td>    <td>                ...</td>    <td>      ...</td></tr>
<tr><td>140255</td> <td>2015-01-01 00:00:00</td>    <td>-0.095751</td></tr>
</table>

### Instructions
-    Iterate over the range of the number of data points minus the length of an input sequence.
-    Define the inputs x as the slice of df from the ith row to the i + seq_lengthth row and the column at index 1.
-    Define the target y as the slice of df at row index i + seq_length and the column at index 1.

In [23]:
import numpy as np

def create_sequences(df, seq_length):
    xs, ys = [], []
    # Iterate over data indices
    for i in range(len(df) - seq_length):
      	# Define inputs
        x = df.iloc[i:(i+seq_length), 1]
        # Define target
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

## Sequential Dataset

##### Good job building the create_sequences() function! It's time to use it to create a training dataset for your model.

##### Just like tabular and image data, sequential data is easiest passed to a model through a torch Dataset and DataLoader. To build a sequential Dataset, you will call create_sequences() to get the NumPy arrays with inputs and targets, and inspect their shape. Next, you will pass them to a TensorDataset to create a proper torch Dataset, and inspect its length.

##### Your implementation of create_sequences() and a DataFrame with the training data called train_data are available.

### Instructions
-    Call create_sequences(), passing it the training DataFrame and a sequence length of 24*4, assigning the result to X_train, y_train.
-    Define dataset_train by calling TensorDataset and passing it two arguments, the inputs and the targets created by create_sequences(), both converted from NumPy arrays to tensors of floats.

In [24]:
train_data = pd.read_csv("electricity_consump/electricity_train.csv")

In [25]:
import torch
from torch.utils.data import TensorDataset

# Use create_sequences to create inputs and targets
X_train, y_train = create_sequences(train_data, 24*4)
print(X_train.shape, y_train.shape)

# Create TensorDataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float(),
)
print(len(dataset_train))

(105119, 96) (105119,)
105119


## Building a forecasting RNN

##### It's time to build your first recurrent network! It will be a sequence-to-vector model consisting of an RNN layer with two layers and a hidden_size of 32. After the RNN layer, a simple linear layer will map the outputs to a single value to be predicted.

##### The following imports have already be done for you:

<code>
import torch
import torch.nn as nn
</code>

### Instructions 1/4
-    Define the RNN layer passing it the correct values for input_size, hidden_size, num_layers, and batch_first, and assign it to self.rnn.

In [26]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size = 32,
            num_layers = 2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

### Instructions 2/4
-    Initialize the first hidden state h0 as a tensor of zeros of the appropriate shape.

In [27]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        # Initialize first hidden state with zeros
        h0 = torch.zeros(2, x.size(0), 32)

### Instructions 3/4
-    Pass the input x and the first hidden state h0 through recurrent layer.

In [28]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        # Initialize first hidden state with zeros
        h0 = torch.zeros(2, x.size(0), 32)
        # Pass x and h0 through recurrent layer
        out, _ = self.rnn(x, h0)

### Instructions 4/4
-    Pass recurrent layer's last output through the linear layer

In [29]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.RNN(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        # Initialize first hidden state with zeros
        h0 = torch.zeros(2, x.size(0), 32)
        # Pass x and h0 through recurrent layer
        out, _ = self.rnn(x, h0)  
        # Pass recurrent layer's last output through linear layer
        out = self.fc(out[:, -1, :])
        return out

## LSTM network

##### As you already know, plain RNN cells are not used that much in practice. A more frequently used alternative that ensures a much better handling of long sequences are Long Short-Term Memory cells, or LSTMs. In this exercise, you will be build an LSTM network yourself!

##### The most important implementation difference from the RNN network you have built previously comes from the fact that LSTMs have two rather than one hidden states. This means you will need to initialize this additional hidden state and pass it to the LSTM cell.

##### torch and torch.nn have already been imported for you, so start coding!

### Instructions
-    In the .\_\_init\_\_() method, define an LSTM layer and assign it to self.lstm.
-    In the forward() method, initialize the first long-term memory hidden state c0 with zeros.
-    In the forward() method, pass all three inputs to the LSTM layer: the current time step's inputs, and a tuple containing the two hidden states.

In [30]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        # Define lstm layer
        self.lstm = nn.LSTM(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        # Initialize long-term memory
        c0 = torch.zeros(2, x.size(0), 32)
        # Pass all inputs to lstm layer
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

## GRU network

##### Next to LSTMs, another popular recurrent neural network variant is the Gated Recurrent Unit, or GRU. It's appeal is in its simplicity: GRU cells require less computation than LSTM cells while often matching them in performance.

##### The code you are provided with is the RNN model definition that you coded previously. Your task is to adapt it such that it produces a GRU network instead. torch and torch.nn as nn have already been imported for you.

### Instructions
-    Update the RNN model definition in order to obtain a GRU network.

In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Define RNN layer
        self.rnn = nn.GRU(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            batch_first=True,
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 32)
        out, _ = self.gru(x, h0)  
        out = self.fc(out[:, -1, :])
        return out

## RNN training loop

###### It's time to train the electricity consumption forecasting model!

###### You will use the LSTM network you have defined previously which is available to you as Net, as is the dataloader_train you built before. You will also need to use torch.nn which has already been imported as nn.

###### In this exercise, you will train the model for only three epochs to make sure the training progresses as expected. Let's get to it!

### Instructions
-    Set up the Mean Squared Error loss and assign it to criterion.
-    Reshape seqs to (batch size, sequence length, num features), which in our case is (16, 96, 1), and re-assign the result to seqs.
-    Pass seqs to the model to get its outputs.
-    Based on previously computed quantities, calculate the loss, assigning it to loss.

In [32]:
# Create a DataLoader based on dataset_train
dataloader_train = DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
)

In [33]:
net = Net()
# Set up MSE loss
criterion = nn.MSELoss()
optimizer = optim.Adam(
  net.parameters(), lr=0.0001
)

for epoch in range(3):
    for seqs, labels in dataloader_train:
        # Reshape model inputs
        seqs = seqs.view(16, 96, 1)
        # Get model outputs
        outputs = net(seqs)
        # Compute loss
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

RuntimeError: shape '[16, 96, 1]' is invalid for input of size 192

## Evaluating forecasting models

###### It's evaluation time! The same LSTM network that you have trained in the previous exercise has been trained for you for a few more epochs and is available as net.

###### Your task is to evaluate it on a test dataset using the Mean Squared Error metric (torchmetrics has already been imported for you). Let's see how well the model is doing!

### Instructions
-    Define the Mean Squared Error metrics and assign it to mse.
-    Pass the input sequence to net, and squeeze the result before you assign it to outputs.
-    Compute the final value of the test metric assigning it to test_mse.

In [ ]:
# Define MSE metric
mse = torchmetrics.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in dataloader_test:
        seqs = seqs.view(32, 96, 1)
        # Pass seqs to net and squeeze the result
        outputs = net(seqs).squeeze()
        mse(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")